In [1]:
# ---------------------------------导入模块------------------------------
import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import jieba.posseg as psg

In [3]:
result_path = 'E:/泰迪杯/数据挖掘2021/LDA结果/酒店.csv'
result_path_gbk = 'E:/泰迪杯/数据挖掘2021/LDA结果/酒店gbk.csv'

In [4]:
# ----------------------------------获取数据------------------------------------
path0 = 'E:/泰迪杯/数据挖掘2021/C/C题数据/清洗过的数据/景区评论utf-8.csv'
reviews = pd.read_csv(path0, encoding='utf-8')['评论内容']

In [5]:
reviews

0        是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...
1        **景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...
2        很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...
3        有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是34...
4        周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票198 需要出示相关证件（校卡或...
                               ...                        
58663    还好吧。我们刚刚到瀑布楼遇到一点小意外，打电话到景区办公室要求帮助，景区值班领导马上行动，在...
58664                                 山高路远，走的很辛苦。景色宜人爬山很累。
58665                             环境很好，空气非常棒，很适合全家旅游，特别是避暑
58666                                  都很方便，价格实惠吧，可以预早就订好。
58667                          旅行社不负责任 到了景点没有与门票售票协调好 等了很久
Name: 评论内容, Length: 58668, dtype: object

# 去除无意义的词语，还有英文、数字、繁体字

In [6]:
str_info = '[a-zA-Z]|[\d]|[\s]'
content = reviews.apply(lambda x: re.sub(str_info, '', x))

In [7]:
content

0        是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...
1        **景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，点才能检票进入，我们点多看看人多起来...
2        很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...
3        有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是分钟...
4        周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票需要出示相关证件（校卡或者录取通...
                               ...                        
58663    还好吧。我们刚刚到瀑布楼遇到一点小意外，打电话到景区办公室要求帮助，景区值班领导马上行动，在...
58664                                 山高路远，走的很辛苦。景色宜人爬山很累。
58665                             环境很好，空气非常棒，很适合全家旅游，特别是避暑
58666                                  都很方便，价格实惠吧，可以预早就订好。
58667                            旅行社不负责任到了景点没有与门票售票协调好等了很久
Name: 评论内容, Length: 58668, dtype: object

# 分词

In [8]:
worker = lambda s:[(x.word, x.flag) for x in psg.cut(s)] # 自定义分词函数
seg_word = content.apply(worker)

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 1.652 seconds.
Prefix dict has been built successfully.


In [9]:
seg_word

0        [(是, v), (亲子, n), (游, v), (的, uj), (绝佳, z), (场...
1        [(*, x), (*, x), (景区, n), (差不多, l), (，, x), (票...
2        [(很, d), (有, v), (*, x), (*, x), (特色, n), (的, ...
3        [(有, v), (园区, n), (的, uj), (工作人员, n), (在, p), ...
4        [(周五, t), (逃课, v), (跟, p), (朋友, n), (在, p), (广...
                               ...                        
58663    [(还好, v), (吧, y), (。, x), (我们, r), (刚刚, d), (到...
58664    [(山高路远, ns), (，, x), (走, v), (的, uj), (很, a), ...
58665    [(环境, n), (很, d), (好, a), (，, x), (空气, n), (非常...
58666    [(都, d), (很, d), (方便, a), (，, x), (价格, n), (实惠...
58667    [(旅行社, n), (不负责任, l), (到, v), (了, ul), (景点, n)...
Name: 评论内容, Length: 58668, dtype: object

# 构建分词后的数据框
## 数据框的列为：所切分的词汇，词所在评论的id（reviews_n表），词语在该句子的位置，词性

In [10]:
# 先看seg_word表中，每条评论词的个数
n_word = seg_word.apply(lambda x: len(x)) # 每一评论中词的个数

In [11]:
n_word

0        131
1         64
2        133
3        174
4        181
        ... 
58663     58
58664     12
58665     16
58666     13
58667     14
Name: 评论内容, Length: 58668, dtype: int64

In [12]:
# 将嵌套的列表展开，作为词所在评论的id(来源于reviews_n的行，也是seg_word表的行标签加1);
n_content = [[x+1]*y for x, y in zip(list(seg_word.index), list(n_word))]
index_content = sum(n_content, [])

In [13]:
index_content

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [14]:
# 将seg_word表展开，单独的词作为一行；并将词语与词性分开；
seg_word = sum(seg_word,[])

In [15]:
seg_word

[('是', 'v'),
 ('亲子', 'n'),
 ('游', 'v'),
 ('的', 'uj'),
 ('绝佳', 'z'),
 ('场所', 'n'),
 ('，', 'x'),
 ('门票', 'n'),
 ('就是', 'd'),
 ('有点', 'n'),
 ('贵', 'a'),
 ('，', 'x'),
 ('不过', 'c'),
 ('可以', 'c'),
 ('接受', 'v'),
 ('，', 'x'),
 ('爷爷奶奶', 'n'),
 ('不', 'd'),
 ('放心', 'v'),
 ('小朋友', 'n'),
 ('也', 'd'),
 ('跟上来', 'v'),
 ('了', 'ul'),
 ('，', 'x'),
 ('当天', 't'),
 ('我们', 'r'),
 ('十点', 'm'),
 ('多', 'm'),
 ('就', 'd'),
 ('到', 'v'),
 ('了', 'ul'),
 ('，', 'x'),
 ('错过', 'v'),
 ('了', 'ul'),
 ('节假日', 't'),
 ('，', 'x'),
 ('人', 'n'),
 ('也', 'd'),
 ('还是', 'c'),
 ('多', 'm'),
 ('，', 'x'),
 ('不过', 'c'),
 ('错峰', 'n'),
 ('出行', 'v'),
 ('我们', 'r'),
 ('一天', 'm'),
 ('是', 'v'),
 ('把', 'p'),
 ('动物园', 'n'),
 ('逛完', 'v'),
 ('了', 'ul'),
 ('，', 'x'),
 ('两种', 'm'),
 ('路线', 'n'),
 ('都', 'd'),
 ('逛', 'v'),
 ('完', 'v'),
 ('了', 'ul'),
 ('，', 'x'),
 ('早上', 't'),
 ('我们', 'r'),
 ('先', 'd'),
 ('坐', 'v'),
 ('的', 'uj'),
 ('缆车', 'n'),
 ('，', 'x'),
 ('缆车', 'n'),
 ('人', 'n'),
 ('多', 'm'),
 ('，', 'x'),
 ('排', 'v'),
 ('了', 'ul'),
 ('半小时', 'b'),
 ('

In [16]:
word = [x[0] for x in seg_word] # 词
nature = [x[1] for x in seg_word] # 词性

In [17]:
# 构建目标数据框
result = pd.DataFrame({'index_content':index_content,
                       'word':word,
                       'nature':nature
                       })

In [18]:
result.head()

,index_content,word,nature
0,1,是,v
1,1,亲子,n
2,1,游,v
3,1,的,uj
4,1,绝佳,z


In [19]:
result.shape

(1577398, 3)

## 删除标点符号，停用词

In [20]:
# 观察数据，先将词性是标点符号的删除；
result = result[result['nature'] != 'x']

In [21]:
# 删除停用词
stop_file = open('E:/泰迪杯/数据挖掘2021/C/停用词表.txt','r',encoding='utf-8')
stop = [item[:-1] for item in stop_file.readlines()]+['\n',"区"]+['\n', '导游']+['\n', '感觉']+['\n', '岛上']+['\n', '感受']+['\n', '山']+['\n', '坐']+['\n', '走']+['\n', '不错']

In [22]:
stop

['\ufeff',
 ' ',
 '说',
 '人',
 '元',
 'hellip',
 '&',
 ',',
 '?',
 '、',
 '。',
 '"',
 '"',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '""',
 '的',
 '了',
 '人民',
 '末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎哟',
 '唉',
 '俺',
 '俺们',
 '按',
 '按照',
 '吧',
 '吧哒',
 '把',
 '罢了',
 '被',
 '本',
 '本着',
 '比',
 '比方',
 '比如',
 '鄙人',
 '彼',
 '彼此',
 '边',
 '别',
 '别的',
 '别说',
 '并',
 '并且',
 '不比',
 '不成',
 '不单',
 '不但',
 '不独',
 '不管',
 '不光',
 '不过',
 '不仅',
 '不拘',
 '不论',
 '不怕',
 '不然',
 '不如',
 '不特',
 '不惟',
 '不问',
 '不只',
 '朝',
 '朝着',
 '趁',
 '趁着',
 '乘',
 '冲',
 '除',
 '除此之外',
 '除非',
 '除了',
 '此',
 '此间',
 '此外',
 '从',
 '从而',
 '打',
 '待',
 '但',
 '但是',
 '当',
 '当着',
 '到',
 '得',
 '的',
 '的话',
 '等',
 '等等',
 '地',
 '第',
 '叮咚',
 '对',
 '对于',
 '多',
 '多少',
 '而',
 '而况',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '尔后',
 '反过来',
 '反过来说',
 '反之',
 '非但',
 '非徒',
 '否则',
 '嘎',
 '嘎登',
 '该',
 '赶',
 '个',
 '各',
 '各个',
 '各位',
 '各种',
 '各自',
 '给',
 '根据',
 '跟',
 '故',
 '故此',
 '固然',
 '关于',
 '管',
 '归',
 '果然',
 '果真',
 '过',
 '哈',
 '哈哈',
 '呵',
 '和',
 '何',
 '何处',
 '何况',
 '何时',
 '嘿',
 '

In [23]:
# 上面已经定义过word = [x[0] for x in seg_word]，在此处理
word_list = list(set(word) - set(stop)) # 去除word列表中的停用词
result = result[result['word'].isin(word_list)]# 掩码提取

In [24]:
result

,index_content,word,nature
1,1,亲子,n
4,1,绝佳,z
5,1,场所,n
10,1,贵,a
14,1,接受,v
...,...,...,...
1577384,58668,旅行社,n
1577385,58668,不负责任,l
1577392,58668,售票,n
1577393,58668,协调,v


## 构建各词在在对应评论的位置列

In [25]:
list(result.groupby(by=['index_content']))

[(1,      index_content  word nature
  1                1    亲子      n
  4                1    绝佳      z
  5                1    场所      n
  10               1     贵      a
  14               1    接受      v
  16               1  爷爷奶奶      n
  18               1    放心      v
  21               1   跟上来      v
  24               1    当天      t
  26               1    十点      m
  32               1    错过      v
  34               1   节假日      t
  42               1    错峰      n
  43               1    出行      v
  48               1   动物园      n
  49               1    逛完      v
  52               1    两种      m
  53               1    路线      n
  55               1     逛      v
  59               1    早上      t
  61               1     先      d
  62               1     坐      v
  64               1    缆车      n
  66               1    缆车      n
  70               1     排      v
  72               1   半小时      b
  73               1     队      n
  76               1    火车      n
  79       

In [26]:
# 在原有的reviews_n表中，每条评论所提取的词汇长度如下：
n_word = list(result.groupby(by=['index_content'])['index_content'].count())

In [27]:
n_word

[49,
 17,
 49,
 63,
 81,
 5,
 11,
 41,
 52,
 35,
 14,
 15,
 23,
 16,
 87,
 47,
 33,
 31,
 2,
 3,
 7,
 41,
 2,
 2,
 79,
 3,
 150,
 52,
 3,
 1,
 99,
 23,
 26,
 11,
 23,
 27,
 9,
 16,
 10,
 16,
 22,
 30,
 27,
 146,
 7,
 19,
 3,
 20,
 36,
 3,
 23,
 5,
 28,
 16,
 3,
 2,
 11,
 4,
 88,
 3,
 28,
 36,
 2,
 69,
 37,
 4,
 7,
 5,
 21,
 5,
 17,
 22,
 3,
 2,
 3,
 2,
 34,
 62,
 9,
 22,
 12,
 10,
 27,
 34,
 15,
 20,
 6,
 44,
 6,
 3,
 4,
 28,
 13,
 3,
 16,
 14,
 6,
 12,
 29,
 7,
 5,
 9,
 27,
 5,
 17,
 21,
 26,
 26,
 5,
 31,
 4,
 2,
 18,
 24,
 5,
 25,
 8,
 9,
 11,
 23,
 10,
 19,
 15,
 65,
 4,
 3,
 39,
 9,
 5,
 5,
 3,
 5,
 4,
 14,
 2,
 5,
 11,
 30,
 5,
 2,
 6,
 16,
 8,
 62,
 4,
 17,
 7,
 3,
 14,
 16,
 16,
 33,
 12,
 5,
 5,
 4,
 19,
 18,
 11,
 1,
 18,
 26,
 12,
 40,
 33,
 2,
 4,
 2,
 2,
 7,
 8,
 20,
 6,
 19,
 6,
 5,
 23,
 42,
 6,
 2,
 2,
 32,
 34,
 31,
 10,
 61,
 6,
 7,
 2,
 17,
 71,
 3,
 6,
 11,
 3,
 2,
 4,
 18,
 6,
 2,
 8,
 3,
 7,
 12,
 10,
 28,
 16,
 67,
 3,
 4,
 41,
 4,
 7,
 2,
 7,
 2,
 13,
 2,
 2,
 4

In [28]:
# 构建index_word列表；
index_word = [list(np.arange(0, y)) for y in n_word]

In [29]:
index_word

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62],


In [30]:
# 展开index_word列表，表示词语在改评论的位置；合并评论id，评论中词的id，词，词性；
index_word = sum(index_word, [])
result['index_word'] = index_word

In [31]:
result[result['index_content']==1]

,index_content,word,nature,index_word
1,1,亲子,n,0
4,1,绝佳,z,1
5,1,场所,n,2
10,1,贵,a,3
14,1,接受,v,4
16,1,爷爷奶奶,n,5
18,1,放心,v,6
21,1,跟上来,v,7
24,1,当天,t,8
26,1,十点,m,9


In [32]:
result.shape

(589531, 4)

In [33]:
path = 'E:/泰迪杯/数据挖掘2021/文本预处理结果/'
result.to_csv(path + '景区utf-8.csv', encoding='utf-8', index=False)
result.to_csv(path + '景区gbk.csv', encoding='gbk', index=False)